# Wikipedia Speedrun Project: Web Scraping!

In [84]:
# import relevant libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import random

#### Initialize important variables and functions


In [85]:
def getLinkData(link):
    Wikidata = requests.get(link)
    htmldata = BeautifulSoup(Wikidata.content, 'html.parser')
    result = htmldata.prettify()
    return result

def checkIfVisited(curlink, storelinks):
    for i in range(len(storelinks)):
        if("https://en.wikipedia.org" + curlink == storelinks[i]):
            return True
    return False



#### write function to recurse through hyperlinks!

In [95]:
storelinks = np.array(np.zeros(1000000), dtype = str)
storelinks = storelinks.astype('U256')
startPage = "https://en.wikipedia.org/wiki/Casiopea"
curPageData = getLinkData(startPage)
numLinks = 0
curIdx = 0
totNumSteps = 50 #control this for testing!
storeStepIndices = np.empty(shape = totNumSteps)
for numSteps in range(totNumSteps):
    links = [i for i in range(len(curPageData)) if curPageData.startswith("<a href=\"/wiki", i)] # isolate each hyperlink
    for i in range(len(links)):
        curlink = curPageData[links[i] + 9 : links[i] + (curPageData[links[i]:links[i]+200]).find('" title')]
        if(curlink and curlink.find(':') < 0 
        #    and not checkIfVisited(curlink, storelinks)
           ): #filter out empty links and internal wikipedia links such as "Directory:" and "Help:"
            storelinks[numLinks] = "https://en.wikipedia.org" + curlink
            numLinks = numLinks + 1
    storeStepIndices[numSteps] = numLinks
    curPageData = getLinkData(storelinks[curIdx])
    randIdx = 1 if (numSteps == 0) else (storeStepIndices[numSteps] - storeStepIndices[numSteps - 1])
    curIdx += random.randrange(0, randIdx)
storelinks = storelinks[storelinks != '0.0']
storelinks2 = []
[storelinks2.append(x + "\n") for x in storelinks if x not in storelinks2]
print(storelinks2)
file1 = open('output1.txt', 'w')
file1.writelines(storelinks2)



C:\Users\yahav\AppData\Local\Temp\ipykernel_31260\3079303089.py:21: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  curIdx += random.randrange(0, randIdx)


['https://en.wikipedia.org/wiki/Issei_Noro\n', 'https://en.wikipedia.org/wiki/Akira_Jimbo\n', 'https://en.wikipedia.org/wiki/Tokyo\n', 'https://en.wikipedia.org/wiki/Jazz_fusion\n', 'https://en.wikipedia.org/wiki/Jazz-funk\n', 'https://en.wikipedia.org/wiki/Smooth_jazz\n', 'https://en.wikipedia.org/wiki/City_pop\n', 'https://en.wikipedia.org/wiki/Alfa_Records\n', 'https://en.wikipedia.org/wiki/Polydor_Records\n', 'https://en.wikipedia.org/wiki/Pony_Canyon\n', 'https://en.wikipedia.org/wiki/Issei_Noro\n', 'https://en.wikipedia.org/wiki/Minoru_Mukaiya\n', 'https://en.wikipedia.org/wiki/Tetsuo_Sakurai\n', 'https://en.wikipedia.org/wiki/Masaaki_Hiyama\n', 'https://en.wikipedia.org/wiki/Akira_Jimbo\n', 'https://en.wikipedia.org/wiki/Cassiopeia_(constellation)\n', 'https://en.wikipedia.org/wiki/Jazz_fusion\n', 'https://en.wikipedia.org/wiki/Issei_Noro\n', 'https://en.wikipedia.org/wiki/Tetsuo_Sakurai\n', 'https://en.wikipedia.org/wiki/Minoru_Mukaiya\n', 'https://en.wikipedia.org/wiki/Randy_B